In [1]:
import pandas as pd
import numpy as np


In [2]:
import cudf
print(cudf.Series([1, 2, 3]))

0    1
1    2
2    3
dtype: int64


In [4]:
import cudf
from cuml.svm import SVR
from cuml.model_selection import train_test_split
from cuml.common.device_selection import using_device_type


In [5]:
df = pd.read_csv('./cleaned_data.csv')

df.drop(['Start_Time', 'End_Time', 'Zipcode'], axis=1, inplace=True)

df['Severity'] = df['Severity'].astype('category')
df['month'] = df['month'].astype('category')
df['year'] = df['year'].astype('category')
df['day_of_week'] = df['day_of_week'].astype('category')
df['Sunrise_Sunset'] = df['Sunrise_Sunset'].astype('category')

df

,Severity,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Sunrise_Sunset,Day_Night,duration_minutes,month,year,day_of_week,zipcode_population
0,3,0.010,37.9,35.5,97.0,29.63,7.0,3.5,0.03,Day,1,30.000000,2,2016,1,64746.0
1,3,0.010,37.4,33.8,100.0,29.62,3.0,4.6,0.02,Day,1,30.000000,2,2016,1,64746.0
2,3,0.010,37.4,33.8,100.0,29.62,3.0,4.6,0.02,Day,1,30.000000,2,2016,1,58916.0
3,2,0.010,37.4,33.8,100.0,29.62,3.0,4.6,0.02,Day,1,30.000000,2,2016,1,36724.0
4,2,0.000,33.8,29.6,100.0,29.62,2.0,4.6,0.01,Day,1,30.000000,2,2016,1,58079.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5227101,2,0.543,86.0,86.0,40.0,28.92,10.0,13.0,0.00,Day,1,28.600000,8,2019,5,22136.0
5227102,2,0.338,70.0,70.0,73.0,29.39,10.0,6.0,0.00,Day,1,26.883333,8,2019,5,24201.0
5227103,2,0.561,73.0,73.0,64.0,29.74,10.0,10.0,0.00,Day,1,28.466667,8,2019,5,14873.0
5227104,2,0.772,71.0,71.0,81.0,29.62,10.0,8.0,0.00,Day,1,29.350000,8,2019,5,32005.0


In [6]:
df = pd.get_dummies(df, columns=['Severity', 'month', 'day_of_week', 'Sunrise_Sunset'], drop_first=True)

In [11]:
len(df)

5227106

In [7]:
df.dropna(inplace=True)

In [8]:
len(df)

5225448

In [9]:
# Assuming df is already a cudf DataFrame with preprocessed features and the target column 'duration_minutes'
X = df.drop(columns=['duration_minutes']).astype('float32')
y = df['duration_minutes'].astype('float32')

X = cudf.DataFrame.from_pandas(X)
y = cudf.Series.from_pandas(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [1]:
from cuml.common.device_selection import set_global_device_type, get_global_device_type

initial_device_type = get_global_device_type()
print('default execution device:', initial_device_type)

default execution device: DeviceType.device


In [ ]:
svr_model = SVR(epsilon=0.1, kernel='rbf')  # You can adjust epsilon and kernel as needed

with using_device_type('gpu'):
# Fit the model to training data
    svr_model.fit(X_train, y_train)

In [ ]:
svr_model.save('svr_model.sav')

In [8]:
y_pred = svr_model.predict(X_test)

# Calculate mean absolute error or other relevant metrics
from cuml.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')

ValueError: Error: cuDF Series has missing/null values, which are not supported by cuML.